# Vcfstats
# What does this do?

### Run rtg-tools vcfstats
- Get list of all variants.gvcf.gz files
- Get list of all rtg_vcfstats.txt files
- Convert each to lists of sample IDs
- Compare lists to find samples that are missing rtg_vcfstats.txt files
- Use make-batch-tsv-from-input-sample.py to generate dsub task file
- Run dsub to generate missing rtg_vcfstats.txt files

### Convert vcfstats result files to CSV format
- Get list of all rtg_vcfstats.txt files
- Get list of all rtg_vcfstats.txt.csv files
- Convert each to lists of sample IDs
- Compare lists to find samples that are missing rtg_vcfstats.txt.csv files
- Use make-batch-tsv-from-input-file.py to generate dsub task file
- Run dsub to generate missing rtg_vcfstats.txt.csv files

### Concatenate CSV Files
- Get list of all rtg_vcfstats.txt.csv files
- Run dsub to concatenate all rtg_vcfstats.txt.csv files

# Code

## 0. Check that environment variables have been loaded correctly

Environment variables are imported from the mvp-profile.sh file. If this echo command does not return anything, try sourcing it from the command console. If any of the values are incorrect, change them in mvp-profile.sh, save it, and source it again.

In [1]:
echo "Date stamp: ${date_stamp}"
echo "Home directory: ${mvp_hub}"
echo "Project: ${mvp_project}"
echo "Bucket: ${mvp_bucket}"
echo "Zone: ${mvp_zone}"

Date stamp: 20180601
Home directory: /Users/jinasong/Work/MVP/mvp-on-gcp
Project: gbsc-gcp-project-mvp
Bucket: gbsc-gcp-project-mvp-group/for-jina
Zone: us-*


# 1. Run rtg-tools vcfstats

#### Create file accounting directory it doesn't already exist

In [3]:
accounting_dir="${mvp_hub}/vcfstats/file-accounting/${date_stamp}"
mkdir -p ${accounting_dir}
dsub_inputs_dir="${mvp_hub}/vcfstats/dsub-inputs"
mkdir -p ${dsub_inputs_dir}

#### Get list of sample IDs for gvcf files that already exist on Google Cloud Storage

In [5]:
gsutil ls gs://${mvp_bucket}/*/Variants/*.vcf.gz \
    > ${accounting_dir}/gs-gvcf-${date_stamp}.txt
    
cut -d'/' -f5 ${mvp_hub}/vcfstats/file-accounting/${date_stamp}/gs-gvcf-${date_stamp}.txt \
    > ${accounting_dir}/gs-gvcf-sample-ids-${date_stamp}.txt

#### Get list sample IDs for vcfstats files that already exist on Google Cloud Storage

In [6]:
gsutil ls gs://${mvp_bucket}/dsub/vcfstats/rtg-tools/objects/*_rtg_vcfstats.txt \
    > ${accounting_dir}/gs-vcfstats-rtg-${date_stamp}.txt
cut -d '/' -f8 ${accounting_dir}/gs-vcfstats-rtg-${date_stamp}.txt \ # Check '-f8' parameter 
    | cut -d'_' -f1 > ${accounting_dir}/gs-vcfstats-rtg-sample-ids-${date_stamp}.txt

CommandException: One or more URLs matched no objects.


#### Get difference between lists of sample IDs to find out which samples are missing vcfstats files

In [7]:
diff --new-line-format="" --unchanged-line-format "" \
    <(sort ${accounting_dir}/gs-gvcf-sample-ids-${date_stamp}.txt) \
    <(sort ${accounting_dir}/gs-vcfstats-rtg-sample-ids-${date_stamp}.txt) \
    > ${accounting_dir}/gs-vcfstats-rtg-missing-sample-ids-${date_stamp}.txt
grep -F \
    -f ${accounting_dir}/gs-vcfstats-rtg-missing-sample-ids-${date_stamp}.txt \
    ${accounting_dir}/gs-gvcf-${date_stamp}.txt \
    > ${accounting_dir}/gs-vcfstats-rtg-missing-${date_stamp}.txt

#### Create dsub TSV task file to generate missing vcfstats files

In [8]:
${mvp_hub}/bin/make-batch-tsv-from-input-sample.py \
    -i ${accounting_dir}/gs-vcfstats-rtg-missing-${date_stamp}.txt \
    -t ${dsub_inputs_dir}/rtg-tools/gs-vcfstats-rtg-missing-${date_stamp}.tsv \
    -o gs://${mvp_bucket}/dsub/vcfstats/rtg-tools/objects \
    -s rtg_vcfstats.txt

#### Run RTG vcfstats dsub tasks

#### Get list of sample IDs for vcfstats CSV files that already exist on Google Cloud Storage

In [9]:
dsub \
    --zones "${mvp_zone}" \
    --project ${mvp_project} \
    --logging gs://${mvp_bucket}/dsub/vcfstats/rtg-tools/logs/${date_stamp} \
    --image gcr.io/${mvp_project}/rtg-tools:1.0 \
    --command 'rtg vcfstats ${INPUT} > ${OUTPUT}' \
    --tasks ${dsub_inputs_dir}/rtg-tools/gs-vcfstats-rtg-missing-${date_stamp}.tsv 

Job: rtg--jinasong--180601-150728-50
Launched job-id: rtg--jinasong--180601-150728-50
1 task(s)
To check the status, run:
  dstat --project gbsc-gcp-project-mvp --jobs 'rtg--jinasong--180601-150728-50' --status '*'
To cancel the job, run:
  ddel --project gbsc-gcp-project-mvp --jobs 'rtg--jinasong--180601-150728-50'
rtg--jinasong--180601-150728-50


## 2. Convert vcfstats result files to CSV format

#### Get list of sample IDs for vcfstats files that already exist on Google Cloud Storage

In [10]:
gsutil ls gs://${mvp_bucket}/dsub/vcfstats/rtg-tools/objects/*_rtg_vcfstats.txt \
    > ${accounting_dir}/gs-vcfstats-rtg-${date_stamp}.txt
cut -d'/' -f9 ${accounting_dir}/gs-vcfstats-rtg-${date_stamp}.txt \
    | cut -d'_' -f1 > ${accounting_dir}/gs-vcfstats-rtg-sample-ids-${date_stamp}.txt

#### Get list of sample IDs for vcfstats CSV files that already exist on Google Cloud Storage

In [11]:
gsutil ls gs://${mvp_bucket}/dsub/vcfstats/text-to-table/objects/*_rtg_vcfstats.txt.csv \
    > ${accounting_dir}/gs-vcfstats-csv-${date_stamp}.txt
cut -d'/' -f8 ${accounting_dir}/gs-vcfstats-csv-${date_stamp}.txt \
    | cut -d'_' -f1 > ${accounting_dir}/gs-vcfstats-csv-sample-ids-${date_stamp}.txt

CommandException: One or more URLs matched no objects.


#### Get difference between lists of sample IDs to find out which samples are missing vcfstats files

In [12]:
diff \
    --new-line-format="" \
    --unchanged-line-format "" \
    <(sort ${accounting_dir}/gs-vcfstats-rtg-sample-ids-${date_stamp}.txt) \
    <(sort ${accounting_dir}/gs-vcfstats-csv-sample-ids-${date_stamp}.txt) \
    > ${accounting_dir}/gs-vcfstats-csv-sample-ids-missing-${date_stamp}.txt
grep -F -f \
    ${accounting_dir}/gs-vcfstats-csv-sample-ids-missing-${date_stamp}.txt \
    ${accounting_dir}/gs-vcfstats-rtg-${date_stamp}.txt \
    > ${accounting_dir}/gs-vcfstats-csv-missing-${date_stamp}.txt

#### Convert file list to dsub TSV files

In [13]:
${mvp_hub}/bin/make-batch-tsv-from-input-file.py \
    -i ${accounting_dir}/gs-vcfstats-csv-missing-${date_stamp}.txt \
    -t ${dsub_inputs_dir}/text-to-table/gs-vcfstats-csv-missing-${date_stamp}.tsv \
    -o gs://${mvp_bucket}/dsub/vcfstats/text-to-table/objects \
    -s csv \
    -c rtg_vcfstats \
    -e rtg-vcfstats-${date_stamp}

#### Run dsub task

In [1]:
dsub \
    --zones ${mvp_zone} \
    --project ${mvp_project} \
    --logging gs://${mvp_bucket}/dsub/vcfstats/text-to-table/logs/${date_stamp} \
    --image gcr.io/${mvp_project}/text-to-table:0.2.0 \
    --command 'text2table -s ${SCHEMA} -o ${OUTPUT} -v series=${SERIES},sample=${SAMPLE_ID} ${INPUT}' \
    --tasks ${mvp_hub}/vcfstats/dsub-inputs/text-to-table/gs-vcfstats-csv-missing-${date_stamp}.tsv

Job: text2table--jinasong--180601-152201-48
Launched job-id: text2table--jinasong--180601-152201-48
1 task(s)
To check the status, run:
  dstat --project gbsc-gcp-project-mvp --jobs 'text2table--jinasong--180601-152201-48' --status '*'
To cancel the job, run:
  ddel --project gbsc-gcp-project-mvp --jobs 'text2table--jinasong--180601-152201-48'
text2table--jinasong--180601-152201-48


## 3. Concatenate CSV Files

#### Get new list of completed results files

In [4]:
gsutil ls gs://${mvp_bucket}/dsub/vcfstats/text-to-table/objects/*_rtg_vcfstats.txt.csv \
    > ${accounting_dir}/gs-vcfstats-csv-${date_stamp}.txt

#### Run dsub task

In [6]:
dsub \
    --zones ${mvp_zone} \
    --project ${mvp_project} \
    --logging gs://${mvp_bucket}/dsub/vcfstats/concat/logs/${date_stamp} \
    --image gcr.io/${mvp_project}/text-to-table:0.2.0 \
    --disk-size 100 \
    --input INPUT_FILES=gs://${mvp_bucket}/dsub/vcfstats/text-to-table/objects/*_vcfstats.txt.csv \
    --output CONCAT_FILE=gs://${mvp_bucket}/dsub/vcfstats/concat/objects/concat_vcfstats.txt.csv \
    --command 'cat ${INPUT_FILES} > ${CONCAT_FILE}' \
    #--dry-run
    #--vars-include-wildcards \

Job: cat--jinasong--180601-160157-53
Launched job-id: cat--jinasong--180601-160157-53
To check the status, run:
  dstat --project gbsc-gcp-project-mvp --jobs 'cat--jinasong--180601-160157-53' --status '*'
To cancel the job, run:
  ddel --project gbsc-gcp-project-mvp --jobs 'cat--jinasong--180601-160157-53'
cat--jinasong--180601-160157-53
